In [24]:
import os, sys, glob
import tensorflow as tf
import segmentation_dataset

In [6]:
tfrecords = glob.glob("../datasets/check_localization/tfrecord/*")

In [8]:
len(tfrecords)

32

In [32]:
slim = tf.contrib.slim

dataset_data_provider = slim.dataset_data_provider


def _get_data(data_provider, dataset_split):
  """Gets data from data provider.

  Args:
    data_provider: An object of slim.data_provider.
    dataset_split: Dataset split.

  Returns:
    image: Image Tensor.
    label: Label Tensor storing segmentation annotations.
    image_name: Image name.
    height: Image height.
    width: Image width.

  Raises:
    ValueError: Failed to find label.
  """
  if 'labels_class' not in data_provider.list_items():
    raise ValueError('Failed to find labels.')

  image, height, width = data_provider.get(
      ['image', 'height', 'width'])

  # Some datasets do not contain image_name.
  if 'image_name' in data_provider.list_items():
    image_name, = data_provider.get(['image_name'])
  else:
    image_name = tf.constant('')

  label = None
  if dataset_split != 'test':
    label, = data_provider.get(['labels_class'])

  return image, label, image_name, height, width

is_training = True
dataset = segmentation_dataset.get_dataset('check_localization', 
                                           'train', 
                                           dataset_dir="../datasets/check_localization/tfrecord")
data_provider = dataset_data_provider.DatasetDataProvider(
      dataset,
      num_readers=1,
      num_epochs=None if is_training else 1,
      shuffle=is_training)

image, label, image_name, height, width = _get_data(data_provider, 'train')

In [38]:
image

<tf.Tensor 'case_6/cond/Merge:0' shape=(?, ?, 3) dtype=uint8>

In [17]:
filename = tf.train.string_input_producer([tfrecords[0]], num_epochs=10)

# Even when reading in multiple threads, share the filename
# queue.
image, annotation = read_and_decode(filename)

# The op for initializing the variables.
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

with tf.Session()  as sess:
    
    sess.run(init_op)
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    # Let's read off 3 batches just for example
    for i in xrange(3):
    
        img, anno = sess.run([image, annotation])

NameError: name 'image_data' is not defined

In [16]:
def _int64_list_feature(values):
  """Returns a TF-Feature of int64_list.

  Args:
    values: A scalar or list of values.

  Returns:
    A TF-Feature.
  """
  if not isinstance(values, collections.Iterable):
    values = [values]

  return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

def _bytes_list_feature(values):
  """Returns a TF-Feature of bytes.

  Args:
    values: A string.

  Returns:
    A TF-Feature.
  """
  def norm2bytes(value):
    return value.encode() if isinstance(value, str) and six.PY3 else value

  return tf.train.Feature(
      bytes_list=tf.train.BytesList(value=[norm2bytes(values)]))

def read_and_decode(filename):
    
    reader = tf.TFRecordReader()

    _, serialized_example = reader.read(filename)

    features = tf.parse_single_example(serialized_example, 
        feature={'image/encoded': _bytes_list_feature(image_data),
                 'image/filename': _bytes_list_feature(filename),
                 'image/format': _bytes_list_feature(
                                    _IMAGE_FORMAT_MAP[FLAGS.image_format]),
                 'image/height': _int64_list_feature(height),
                 'image/width': _int64_list_feature(width),
                 'image/channels': _int64_list_feature(3),
                 'image/segmentation/class/encoded': (_bytes_list_feature(seg_data)),
                 'image/segmentation/class/format': _bytes_list_feature(FLAGS.label_format),
                })

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['image/encoded'], tf.uint8)
    annotation = tf.decode_raw(features['image/segmentation/class/encoded'], tf.uint8)
    
    height = tf.cast(features['image/height'], tf.int32)
    width = tf.cast(features['image/width'], tf.int32)
    
    image_shape = tf.pack([height, width, 3])
    annotation_shape = tf.pack([height, width, 1])
    
    image = tf.reshape(image, image_shape)
    annotation = tf.reshape(annotation, annotation_shape)
    
#     image_size_const = tf.constant((IMAGE_HEIGHT, IMAGE_WIDTH, 3), dtype=tf.int32)
#     annotation_size_const = tf.constant((IMAGE_HEIGHT, IMAGE_WIDTH, 1), dtype=tf.int32)
    
    # Random transformations can be put here: right before you crop images
    # to predefined size. To get more information look at the stackoverflow
    # question linked above.
    
#     resized_image = tf.image.resize_image_with_crop_or_pad(image=image,
#                                            target_height=IMAGE_HEIGHT,
#                                            target_width=IMAGE_WIDTH)
    
#     resized_annotation = tf.image.resize_image_with_crop_or_pad(image=annotation,
#                                            target_height=IMAGE_HEIGHT,
#                                            target_width=IMAGE_WIDTH)
    
    
#     images, annotations = tf.train.shuffle_batch( [resized_image, resized_annotation],
#                                                  batch_size=2,
#                                                  capacity=30,
#                                                  num_threads=2,
#                                                  min_after_dequeue=10)
    
    return image, annotation